In [1]:
!pip install sentence_transformers==2.7.0
!pip install pandas
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.3 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
v = model.encode('where can i travel in sydney?')

In [5]:
len(v)

384

In [6]:
import pandas as pd
df = pd.read_csv("/content/travel_data.csv")

In [7]:
df.insert(0, 'id', range(1, len(df) + 1))
df.head()

,id,destination,user_review,travel_tip,rating,best_time_to_visit,local_cuisine_highlights,location_coordinates,popular_attractions,transportation_options,language_spoken,safety_rating,activities_available,cultural_highlights
0,1,"paris, france",Visiting the Eiffel Tower was breathtaking! Th...,Book tickets online to avoid long lines!,5,"April to June, September","Croissants, Macarons","48.8584° N, 2.2945° E","Louvre, Notre-Dame, Montmartre","Metro, Buses, Walking",French,4,"Seine River Cruises, Art Tours","Fashion, Art, Romantic Spots"
1,2,"tokyo, japan",Shibuya Crossing is a must-see. It's an incred...,Visit in the evening for the best lights!,4,"March to May, October","Sushi, Ramen","35.6586° N, 139.7012° E","Senso-ji, Tokyo Tower","Train, Metro, Buses",Japanese,5,"Temple Visits, Food Tours","Technology, Anime Culture"
2,3,"new_york, usa",Central Park is a great escape from the city h...,Rent a bike to explore more of the park.,5,Spring and Fall,"Bagels, Cheesecake","40.7851° N, 73.9683° W","Statue of Liberty, Times Square","Subway, Buses, Taxis",English,4,"Broadway Shows, City Tours","Diverse Cultures, Art Scene"
3,4,"rome, italy",The Colosseum is stunning! Make sure to take a...,Visit early in the morning to avoid crowds.,5,"April to June, September","Pasta Carbonara, Gelato","41.8902° N, 12.4923° E","Vatican City, Pantheon","Metro, Buses, Walking",Italian,4,"Historical Tours, Cooking Classes","Ancient History, Religion"
4,5,"cape_town, south_africa",Table Mountain offers the best views of the ci...,Take the cable car if you’re short on time.,4,November to February,"Bobotie, Biltong","-33.9625° S, 18.4095° E","Cape of Good Hope, Robben Island","Car, Taxi, Walking","Afrikaans, English",4,"Wine Tasting, Hiking","Cultural Heritage, Nature"


In [8]:
documents = df.to_dict(orient='records')
documents[0]

{'id': 1,
 'destination': 'paris, france',
 'user_review': 'Visiting the Eiffel Tower was breathtaking! The views from the top are unforgettable.',
 'travel_tip': 'Book tickets online to avoid long lines!',
 'rating': 5,
 'best_time_to_visit': 'April to June, September',
 'local_cuisine_highlights': 'Croissants, Macarons',
 'location_coordinates': '48.8584° N, 2.2945° E',
 'popular_attractions': 'Louvre, Notre-Dame, Montmartre',
 'transportation_options': 'Metro, Buses, Walking',
 'language_spoken': 'French',
 'safety_rating': 4,
 'activities_available': 'Seine River Cruises, Art Tours',
 'cultural_highlights': 'Fashion, Art, Romantic Spots'}

In [9]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://54.144.217.33:9200')

es_client.info()

ObjectApiResponse({'name': '020cfa9019a5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'U--Sn6YpSTmhWg7qQtl38w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [21]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
          "id": {"type": "integer"},
            "destination": {"type": "text"},
            "user_review": {"type": "text"},
            "travel_tip": {"type": "text"},
            "rating": {"type": "integer"},
            "best_time_to_visit": {"type": "text"},
            "local_cuisine_highlights": {"type": "text"},
            "location_coordinates": {"type": "text"},
            "popular_attractions": {"type": "text"},
            "transportation_options": {"type": "text"},
            "language_spoken": {"type": "text"},
            "safety_rating": {"type": "integer"},
            "activities_available": {"type": "text"},
            "cultural_highlights": {"type": "text"},
            "destination_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "travel_tip_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "user_review_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "best_time_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "local_cuisine_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "popular_attraction_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "transportation_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "language_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "activities_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "cultural_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
          "combined_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"}
        }
    }
}


In [22]:
index_name = "travel-guide-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'travel-guide-questions'})

In [23]:
from tqdm.auto import tqdm

In [24]:
for doc in tqdm(documents):
    destination = doc["destination"]
    user_review = doc["user_review"]
    travel_tip = doc["travel_tip"]
    best_time_to_visit = doc["best_time_to_visit"]
    local_cuisine_highlights = doc["local_cuisine_highlights"]
    location_coordinates = doc["location_coordinates"]
    popular_attractions = doc["popular_attractions"]
    transportation_options = doc["transportation_options"]
    language_spoken = doc["language_spoken"]
    activities_available = doc["activities_available"]
    cultural_highlights = doc["cultural_highlights"]

    combined = destination + ' ' + user_review + ' ' + travel_tip + ' ' + best_time_to_visit + ' ' + local_cuisine_highlights + ' ' + location_coordinates + ' ' + popular_attractions + ' ' + transportation_options + ' ' + language_spoken + ' ' + activities_available + ' ' + cultural_highlights

    doc['destination_vector'] = model.encode(destination)
    doc['user_review_vector'] = model.encode(user_review)
    doc['travel_tip_vector'] = model.encode(travel_tip)
    doc['best_time_to_visit_vector'] = model.encode(best_time_to_visit)  # Adjusted to match mapping
    doc['local_cuisine_highlights_vector'] = model.encode(local_cuisine_highlights)  # Adjusted to match mapping
    doc['location_coordinates_vector'] = model.encode(location_coordinates)
    doc['popular_attractions_vector'] = model.encode(popular_attractions)
    doc['transportation_options_vector'] = model.encode(transportation_options)
    doc['language_spoken_vector'] = model.encode(language_spoken)
    doc['activities_available_vector'] = model.encode(activities_available)
    doc['cultural_highlights_vector'] = model.encode(cultural_highlights)
    doc['combined_vector'] = model.encode(combined)

    es_client.index(index=index_name, body=doc)



  0%|          | 0/215 [00:00<?, ?it/s]

In [25]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/215 [00:00<?, ?it/s]

In [26]:
query = 'where can i travel in sydney?'

In [27]:
v_q = model.encode(query)

In [29]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": [    "id",
                        "destination",
                        "user_review",
                        "travel_tip",
                        "best_time_to_visit",
                        "local_cuisine_highlights",
                        "location_coordinates",
                        "popular_attractions",
                        "transportation_options",
                        "language_spoken",
                        "activities_available",
                        "cultural_highlights"
                    ],
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [30]:
def question_vector_knn(q):
    question = q['question']
    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q)

In [31]:
import pandas as pd

In [40]:
df_ground_truth = pd.read_csv('/content/ground-truth-retrieval.csv')
df_ground_truth.tail()

,id,question
1070,215,What are the best months to visit doha for opt...
1071,215,What popular museums can i find in doha?
1072,215,"I'm visiting doha in May, what tip can you pro..."
1073,215,Are there any ancient cultural heritage sites ...
1074,215,What public transportation options are availab...


In [33]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [34]:
ground_truth[0]

{'id': 1,
 'question': 'What are some of the must-visit museums in Paris that house famous works of art?'}

In [36]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [37]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [41]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [44]:
def destination_vector_knn(q):
    question = q['question']
    v_q = model.encode(question)

    return elastic_search_knn('destination_vector', v_q)

evaluate(ground_truth, destination_vector_knn)

  0%|          | 0/1075 [00:00<?, ?it/s]

{'hit_rate': 0.8493023255813954, 'mrr': 0.8611162790697722}

In [45]:
def combined_vector(q):
    question = q['question']
    v_q = model.encode(question)

    return elastic_search_knn('combined_vector', v_q)

evaluate(ground_truth, destination_vector_knn)

  0%|          | 0/1075 [00:00<?, ?it/s]

{'hit_rate': 0.7637209302325582, 'mrr': 0.8386046511627935}